In [ ]:
import lightgbm as lgb
import wandb
import pandas as pd


run = wandb.init()
artifact = run.use_artifact('andrea-faviait/swiss-delay-prediction/model_o8eefm0p:v0', type='model')
artifact_dir = artifact.download()
model = lgb.Booster(model_file=artifact_dir+'/model_ckpt_0.txt')

wandb: Network error (ConnectionError), entering retry loop.


In [ ]:
test_df = pd.read_csv('/Users/favea/Downloads/swiss-data/test_df.csv' ,index_col=0)
test_df = test_df.drop(['delay_minutes','scheduled_time_departure','year'],axis=1)
x_test, y_test = test_df.drop('delayed',axis=1), test_df.loc[:,'delayed']
x_test.carrier = x_test.carrier.astype('category')
x_test.origin = x_test.origin.astype('category')
x_test.destination = x_test.destination.astype('category')
x_test.previous_is_delayed = x_test.previous_is_delayed.astype('category')

lgb_train = lgb.Dataset(x_test, y_test, categorical_feature = ['carrier','origin','destination','previous_is_delayed'], free_raw_data=False)

In [ ]:
d = {'carrier': 'string', 'origin': 'string', 'destination': 'string', 'distance_trip': 0, 'mails_data': 0, 'cargo_data': 0, 'number_checked_luggages': 0, 'flights_per_day': 0, 'previous_is_delayed': True, 'day_of_week': 0, 'day_of_year': 0, 'month': 0, 'total_number_passengers': 0, 'flights_within_hour': 0}

In [ ]:
pd.DataFrame([d]).T

In [ ]:
x_test.columns

In [ ]:
lgb.plot_importance(model)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
#plt.figure(figsize=(12,12)) 
#lgb.plot_tree(model)
#plt.savefig('tree_high_dpi', dpi=5000)
#plt.show()

In [ ]:
y_test_pred = model.predict(x_test.drop('Unnamed: 0',1))

In [ ]:
x_test['predicted'] = (y_test_pred>=0.5).astype(int)
x_test['true'] = y_test

In [ ]:
x_test.sample(100)

In [ ]:
x_test.groupby(by=['true','predicted']).count().iloc[:,:1]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(x_test['true'],x_test['predicted']))